<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [42]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [43]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [44]:
%%sql
SELECT  m_title
FROM    movies

 * sqlite:///movies.sqlite
Done.


m_title
Batman Begins
Demon Slayer: Mugen Train
Monty Python and the Holy Grail
Snatch
The Lord of the Rings: The Fellowship of the Ring


+ Show the performance dates for one of the movies.

In [45]:
%%sql
SELECT  start_date
FROM    screenings
WHERE   m_title = "Batman Begins"

 * sqlite:///movies.sqlite
Done.


start_date
2022-02-01


+ Show all data concerning performances at a given theatere
  on a given date.

In [46]:
%%sql
SELECT  *
FROM    screenings
WHERE   t_name = 'Filmstaden Lund' AND start_date = '2022-02-01'

 * sqlite:///movies.sqlite
Done.


screening_id,start_time,start_date,free_seats,t_name,m_title,production_year
1,19:30,2022-02-01,100,Filmstaden Lund,Demon Slayer: Mugen Train,2020


+ List all customers

In [47]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.sqlite
Done.


username,full_name,pwd,c_name
marcusoft,marcusbegic,kingen123,Filmstaden
stifi,dylanfrost,somethingstrange,Filmstaden
axley,axelbengtsson,mari6969,Filmstaden
Greven,FredrikHornDannertAfAminne,pleb,Filmstaden
Sollerito,MaxSoller,bayern,Filmstaden
MrFiskpinnar,HenkeRasmusson,obiOneknobe,Filmstaden


+ List all tickets

In [48]:
%%sql
SELECT  *
FROM    tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,screening_id,username
95ad6855e300f9bc98fbe969f987f4c2,1,marcusoft
92023615c0b7b3c7fbbabd2bb7f2d5e6,1,axley
a92d8e0c9bb47111f81c0b33c5adff13,2,marcusoft
9782f6d3924098ba613ea2e61d71b843,2,stifi
0e0ee30095e2dba90b7f50e4aa88318b,3,Greven
e11d81c4d58d6a80619c80787dacb3ba,3,axley
8c88663101efaf911e4c1fa5e8918bb5,3,sollerito
22316eb8c23b95d967a456d9e93e64da,3,MrFiskpinnar
921bde5e1ad98bc83b7219eb904a6ca6,4,marcusoft
d7b54dd4781d48944182fef1ae3f1d0a,4,Sollerito


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [49]:
%%sql
INSERT
INTO    tickets(screening_id, username)
VALUES  (1, 'MrFiskpinnar');

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [50]:
%%sql
SELECT  ticket_id
FROM    tickets
WHERE   rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


ticket_id
5bfe7a9024e4266071b906099a132932


+ Try to insert two movie theaters with the same name (this
  should fail).

In [56]:
%%sql
INSERT
INTO    theaters(t_name, capacity)
VALUES  ('Kino', 100),
        ('Kino', 200);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.t_name
[SQL: INSERT INTO theaters(t_name, capacity)
VALUES  ('Kino', 100),
        ('Kino', 200);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [58]:
%%sql
INSERT
INTO    screenings(start_date, start_time, t_name, free_seats, m_title, production_year)
VALUES  ('2022-02-01', '19:30', 'Kino', 100, 'Demon Slayer: Mugen Train', 2020)


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO screenings(start_date, start_time, t_name, free_seats, m_title, production_year)
VALUES  ('2022-02-01', '19:30', 'Kino', 100, 'Demon Slayer: Mugen Train', 2020)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [59]:
%%sql
INSERT
INTO    tickets(screening_id, username)
VALUES  (5, 'Sollerito')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets(screening_id, username)
VALUES  (5, 'Sollerito')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)